In [1]:
# Install a conda package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install flickrapi


In [2]:
from flickrapi import FlickrAPI
import pandas as pd
#import sys



In [3]:
#set "key" and "secret" in a local configuration file named flickr_APIkey.txt
#for an example see: flickr_APIkey_example.txt
#read-in key and secret from configuration file
import os
print(os.getcwd())
file=open(file="flickr_APIkey.txt", mode="r") #mode r = read the file
key=file.readline()
key=key.rstrip() # Remove all trailing whitespace (e.g., \n)
secret=file.readline()
secret=secret.rstrip()

/Users/KGthatsme/Analyses/FlickrFish


In [4]:
def get_urls(image_tag, MAX_COUNT):    
    flickr = FlickrAPI(key, secret)
    photos = flickr.walk(text=image_tag,
                         tag_mode='all',
                         #tags=image_tag, #use text parameter instead, which searches title, description, and tags
                         bbox='-71.791037, 41.217233, -71.095148, 42.007849', # Narraganset Bay + RI Sound
                         has_geo=1,
                         #lat=41.604168,
                         #lon=-71.320683,
                         extras='date_upload, date_taken, geo, url_o',
                         per_page=50,
                         sort='relevance'
                        ) 
    count=0
    urls=[]
    uploads=[]
    geos=[]
    takens=[]
    for photo in photos:
        if count< MAX_COUNT:
            count=count+1
            print("Fetching url for image number {}".format(count))
            try:
                url=photo.get('url_o')
                #print(url)
                urls.append(url)
                upload=photo.get('date_upload')
                uploads.append(upload)
                taken=photo.get('date_taken')
                takens.append(taken)
                geo=photo.get('geo')
                print(geo)
                geos.append(geo)
            except:
                print("Url for image number {} could not be fetched".format(count))
        else:
            print("Done fetching urls, fetched {} urls out of {}".format(len(urls),MAX_COUNT))
            break
    urls=pd.Series(urls)
    uploads=pd.Series(uploads)
    geos=pd.Series(geos)
    takens=pd.Series(takens)
    print("Writing out the urls in the current directory")
    urls.to_csv(image_tag+"_urls.csv")
    uploads.to_csv(image_tag+"_uploads.csv")
    takens.to_csv(image_tag+"_takens.csv")
    geos.to_csv(image_tag+"_geos.csv")
    print("Done!!!")



In [5]:
# change wd to RESULTS folder
import os
os.chdir("/Users/KGthatsme/Analyses/FlickrFish/RESULTS")

In [6]:
image_tag="fish"
MAX_COUNT=100
get_urls(image_tag=image_tag, MAX_COUNT=MAX_COUNT)

Fetching url for image number 1
None
Fetching url for image number 2
None
Fetching url for image number 3
None
Fetching url for image number 4
None
Fetching url for image number 5
None
Fetching url for image number 6
None
Fetching url for image number 7
None
Fetching url for image number 8
None
Fetching url for image number 9
None
Fetching url for image number 10
None
Fetching url for image number 11
None
Fetching url for image number 12
None
Fetching url for image number 13
None
Fetching url for image number 14
None
Fetching url for image number 15
None
Fetching url for image number 16
None
Fetching url for image number 17
None
Fetching url for image number 18
None
Fetching url for image number 19
None
Fetching url for image number 20
None
Fetching url for image number 21
None
Fetching url for image number 22
None
Fetching url for image number 23
None
Fetching url for image number 24
None
Fetching url for image number 25
None
Fetching url for image number 26
None
Fetching url for imag

In [ ]:
## Only need the following code if running this from Terminal
#def main(): 
#    tag=sys.argv[1]
#    MAX_COUNT=int(sys.argv[2])
#    get_urls(tag,MAX_COUNT)


#if __name__=='__main__': 
#    main() 
# if running the module as the main program (e.g., in terminal: python getFishDat.py): execute function (main)
# if module is imported by another program, _name_ is assigned to the module name "getFishDat": skip function (main)

import csv
import requests
import os
import sys
import time
def put_images(image_tag):
    urls=[]
    FILE_NAME=image_tag+"_urls.csv"
    with open(FILE_NAME,newline="") as csvfile:
        doc=csv.reader(csvfile,delimiter=",")
        for row in doc:
            if row[1].startswith("https"):
                urls.append(row[1])
    if not os.path.isdir(os.path.join(os.getcwd(),FILE_NAME.split("_")[0])):
        os.mkdir(FILE_NAME.split("_")[0])
    t0=time.time()
    for url in enumerate(urls):
        print("Starting download {} of ".format(url[0]+1),len(urls))
        try:
            resp=requests.get(url[1],stream=True)
            path_to_write=os.path.join(os.getcwd(),FILE_NAME.split("_")[0],url[1].split("/")[-1])
            outfile=open(path_to_write,'wb')
            outfile.write(resp.content)
            outfile.close()
            print("Done downloading {} of {}".format(url[0]+1,len(urls)))
        except:
            print("Failed to download url number {}".format(url[0]))
    t1=time.time()
    print("Done with download, job took {} seconds".format(t1-t0))





In [ ]:
put_images(image_tag=image_tag)